Installing dependencies

In [1]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [2]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [3]:
import os
from datetime import datetime

import numpy as np
import pandas as pd
import requests
from binance.client import Client

In [4]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']
ta_api = os.environ['taapi_key']


# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
# info = client.get_symbol_info('ETHUSDT')
# print(info)

In [8]:
def get_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '5m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667394000000,1556.31000000,1557.38000000,1414.70000000,1553.45000000
1,1667397600000,1553.25000000,1572.20000000,1523.86000000,1560.21000000
2,1667401200000,1560.22000000,1561.57000000,1481.81000000,1558.54000000
3,1667404800000,1558.51000000,1574.85000000,1550.09000000,1571.23000000
4,1667408400000,1570.95000000,1577.10000000,1550.91000000,1558.57000000
...,...,...,...,...,...
80,1667682000000,1631.45000000,1633.24000000,1130.86000000,1632.49000000
81,1667685600000,1632.52000000,1634.77000000,1629.07000000,1632.31000000
82,1667689200000,1632.31000000,1634.84000000,1128.68000000,1626.82000000
83,1667692800000,1626.44000000,1633.37000000,1616.39000000,1629.29000000


In [ ]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

# BACKTESTING

Backtesting completed on 6 month historical data on 5m time frame

In [25]:
def get_six_month_data_frame(symbol):
    # starttime = '6 months ago UTC'
    starttime = '1 Jan, 2017'
    interval = '5m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [26]:
six_month_eth_df_raw

,date,open,high,low,close
0,1667395500000,1556.31000000,1557.04000000,1555.87000000,1555.87000000
1,1667395800000,1556.01000000,1557.21000000,1548.24000000,1551.88000000
2,1667396100000,1551.89000000,1555.77000000,1548.22000000,1553.73000000
3,1667396400000,1554.06000000,1557.38000000,1414.70000000,1554.02000000
4,1667396700000,1553.85000000,1556.00000000,1550.99000000,1551.55000000
...,...,...,...,...,...
1010,1667698500000,1627.40000000,1629.22000000,1627.40000000,1628.18000000
1011,1667698800000,1628.20000000,1630.67000000,1628.20000000,1628.61000000
1012,1667699100000,1628.51000000,1628.61000000,1625.26000000,1625.93000000
1013,1667699400000,1625.81000000,1775.71000000,1616.00000000,1618.39000000


In [28]:
def test_macd_logic(symbol):

    symbol_df = get_six_month_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)
    
    tr_list = [(symbol_df['high'] - symbol_df['low']), (symbol_df['low'] - symbol_df['close'].shift()), (symbol_df['high'] - symbol_df['close'].shift())]

    symbol_df['TR1'] = symbol_df['high'] - symbol_df['low']
    symbol_df['prev close'] = symbol_df['close'].shift()
    symbol_df['TR2'] = symbol_df['low'] - symbol_df['close'].shift()
    symbol_df['TR3'] = symbol_df['high'] - symbol_df['close'].shift()
    # symbol_df['TR LIST'] = symbol_df.apply(lambda row : [(row['high'] - row['low']), (row['low'] - row['close'].shift()), (row['high'] - row['close'].shift())])
    # symbol_df['TR LIST'] = symbol_df.apply(symbol_df)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return symbol_df

test_df = test_macd_logic('ETHUSDT')

In [29]:
test_df

,date,open,high,low,close,macd,signal,200_ema,Trigger,TR1,prev close,TR2,TR3
200,2022-11-03 06:05:00,1550.07,2000.00,1547.01,1547.24,0.092202,0.708885,1544.433600,0,452.99,1550.07,-3.06,449.93
201,2022-11-03 06:10:00,1547.01,1548.05,1546.73,1547.64,-0.175890,0.531930,1544.465505,0,1.32,1547.24,-0.51,0.81
202,2022-11-03 06:15:00,1547.22,1548.75,1546.78,1547.45,-0.399085,0.345727,1544.495201,0,1.97,1547.64,-0.86,1.11
203,2022-11-03 06:20:00,1547.45,1547.75,1546.77,1546.83,-0.618864,0.152809,1544.518433,0,0.98,1547.45,-0.68,0.30
204,2022-11-03 06:25:00,1546.85,1547.48,1546.52,1547.29,-0.747308,-0.027214,1544.546011,0,0.96,1546.83,-0.31,0.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010,2022-11-06 01:35:00,1627.40,1629.22,1627.40,1628.18,0.029813,0.078927,1633.406566,0,1.82,1627.41,-0.01,1.81
1011,2022-11-06 01:40:00,1628.20,1630.67,1628.20,1628.61,0.007195,0.064580,1633.358839,0,2.47,1628.18,0.02,2.49
1012,2022-11-06 01:45:00,1628.51,1628.61,1625.26,1625.93,-0.224397,0.006785,1633.284920,0,3.35,1628.61,-3.35,0.00
1013,2022-11-06 01:50:00,1625.81,1775.71,1616.00,1618.39,-1.004768,-0.195526,1633.136712,0,159.71,1625.93,-9.93,149.78


In [30]:
atr_indicator = "atr"
atr_endpoint = f"https://api.taapi.io/{atr_indicator}"

live_price_indicator = "price"
live_price_endpoint = f"https://api.taapi.io/{atr_indicator}"

stop_loss = 0
tp = 0
entry = 0

parameters = {
    'secret': ta_api,
    'exchange': 'binance',
    'symbol': 'ETH/USDT',
    'interval': '1H'
    } 

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

atr_test = 16.74326363683623

for index, row in test_df.iterrows():
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True

            stop_loss = row['close'] - atr_test
            tp = row['close'] + atr_test
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr_test))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

STOP LOSS: 1535.2267363631638, CLOSE: 1533.35, ENTRY: 1551.97
Losses: 16.743263636836218

STOP LOSS: 1528.4267363631639, CLOSE: 1527.31, ENTRY: 1545.17
Losses: 33.486527273672436

TP1: 1614.1632636368363, CLOSE: 1615.94, ENTRY: 1597.42
Profit: 8.371631818418109

TP2: 1630.9065272736725, CLOSE: 1635.62, ENTRY: 1597.42
Profit: 25.114895455254327

TP1: 1639.363263636836, CLOSE: 1644.17, ENTRY: 1622.62
Profit: 33.486527273672436

TP2: 1656.1065272736723, CLOSE: 1656.36, ENTRY: 1622.62
Profit: 50.229790910508655

STOP LOSS: 1633.6667363631639, CLOSE: 1632.91, ENTRY: 1650.41
Losses: 50.229790910508655

wins: 2, profit: $50.229790910508655
losses: 3, losses: $50.229790910508655
P/L: 0.0


In [31]:
eth_raw_df_2 = get_six_month_data_frame('ETHUSDT')
eth_raw_df_2

,date,open,high,low,close
0,1667395500000,1556.31000000,1557.04000000,1555.87000000,1555.87000000
1,1667395800000,1556.01000000,1557.21000000,1548.24000000,1551.88000000
2,1667396100000,1551.89000000,1555.77000000,1548.22000000,1553.73000000
3,1667396400000,1554.06000000,1557.38000000,1414.70000000,1554.02000000
4,1667396700000,1553.85000000,1556.00000000,1550.99000000,1551.55000000
...,...,...,...,...,...
1010,1667698500000,1627.40000000,1629.22000000,1627.40000000,1628.18000000
1011,1667698800000,1628.20000000,1630.67000000,1628.20000000,1628.61000000
1012,1667699100000,1628.51000000,1628.61000000,1625.26000000,1625.93000000
1013,1667699400000,1625.81000000,1775.71000000,1616.00000000,1618.39000000


In [44]:
def test_logic_2(raw_df):
    raw_df['open'] = pd.to_numeric(raw_df['open'])
    raw_df['high'] = pd.to_numeric(raw_df['high'])
    raw_df['low'] = pd.to_numeric(raw_df['low'])
    raw_df['close'] = pd.to_numeric(raw_df['close'])
    raw_df['date'] = pd.to_datetime(raw_df['date'], unit = 'ms')


    short_ema = raw_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = raw_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = raw_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    raw_df['macd'] = macd
    raw_df['signal'] = signal
    raw_df['200_ema'] = ema_200

    raw_df['TR1'] = abs(raw_df['high'] - raw_df['low'])
    raw_df['TR2'] = abs(raw_df['low'] - raw_df['close'].shift())
    raw_df['TR3'] = abs(raw_df['high'] - raw_df['close'].shift())

    raw_df['ATR'] = raw_df.apply(lambda x: row_check(x.TR1, x.TR2, x.TR3), axis=1)

    raw_df = raw_df.iloc[200:]

    raw_df = raw_df.drop(['TR1', 'TR2', 'TR3'], axis=1)

    raw_df['Trigger'] = np.where(((raw_df['macd'] > raw_df['signal']) & (raw_df['close'] > raw_df['200_ema']) & (raw_df['macd'] < 0) & (raw_df['signal'] > raw_df['macd'].shift())), 1, 0)

    return raw_df

def row_check(TR1, TR2, TR3):
    if TR1 >= TR2 and TR1 >= TR3:
        ATR = TR1
    elif TR2 >= TR1 and TR2 >= TR3:
        ATR = TR2
    else:
        ATR = TR3
    
    return ATR

In [45]:
tinkered_df = test_logic_2(eth_raw_df_2)

In [46]:
tinkered_df

,date,open,high,low,close,macd,signal,200_ema,ATR,Trigger
200,2022-11-03 06:05:00,1550.07,2000.00,1547.01,1547.24,0.092202,0.708885,1544.433600,452.99,0
201,2022-11-03 06:10:00,1547.01,1548.05,1546.73,1547.64,-0.175890,0.531930,1544.465505,1.32,0
202,2022-11-03 06:15:00,1547.22,1548.75,1546.78,1547.45,-0.399085,0.345727,1544.495201,1.97,0
203,2022-11-03 06:20:00,1547.45,1547.75,1546.77,1546.83,-0.618864,0.152809,1544.518433,0.98,0
204,2022-11-03 06:25:00,1546.85,1547.48,1546.52,1547.29,-0.747308,-0.027214,1544.546011,0.96,0
...,...,...,...,...,...,...,...,...,...,...
1010,2022-11-06 01:35:00,1627.40,1629.22,1627.40,1628.18,0.029813,0.078927,1633.406566,1.82,0
1011,2022-11-06 01:40:00,1628.20,1630.67,1628.20,1628.61,0.007195,0.064580,1633.358839,2.49,0
1012,2022-11-06 01:45:00,1628.51,1628.61,1625.26,1625.93,-0.224397,0.006785,1633.284920,3.35,0
1013,2022-11-06 01:50:00,1625.81,1775.71,1616.00,1618.39,-1.004768,-0.195526,1633.136712,159.71,0


In [47]:
stop_loss = 0
tp = 0
entry = 0

in_trade = False
tp1_hit = False

win = 0
loss = 0
profit = 0
losses = 0

for index, row in tinkered_df.iterrows():
    atr = 0
    
    if in_trade == False:
        if row['Trigger'] == 1:
            in_trade = True
            atr = row['ATR']
            stop_loss = row['close'] - atr
            tp = row['close'] + atr
            entry = row['close']
            
    
    else:
        if row['close'] <= stop_loss:
            in_trade = False
            close = row['close']
            loss += 1
            losses += (entry - stop_loss)
            tp1_hit = False
            print(f'STOP LOSS: {stop_loss}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Losses: {losses}')
            print('')

        elif row['close'] >= tp and tp1_hit == False:
            # in_trade = False
            close = row['close']
            tp1_hit = True
            stop_loss = entry
            win += 1
            profit += ((tp - entry)/2)
            print(f'TP1: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')
            tp = (entry + (2*atr))
            
        elif row['close'] >= tp and tp1_hit == True:
            close = row['close']
            in_trade = False
            tp1_hit = False
            profit += ((tp - entry)/2)
            print(f'TP2: {tp}, CLOSE: {close}, ENTRY: {entry}')
            print(f'Profit: {profit}')
            print('')


        
print(f'wins: {win}, profit: ${profit}')
print(f'losses: {loss}, losses: ${losses}')
print(f'P/L: {profit - losses}')

STOP LOSS: 1547.88, CLOSE: 1546.81, ENTRY: 1551.97
Losses: 4.089999999999918

STOP LOSS: 1546.84, CLOSE: 1546.2, ENTRY: 1548.36
Losses: 5.6099999999999

TP1: 1601.24, CLOSE: 1608.5, ENTRY: 1550.21
Profit: 25.514999999999986

TP2: 1550.21, CLOSE: 1613.69, ENTRY: 1550.21
Profit: 25.514999999999986

STOP LOSS: 1619.99, CLOSE: 1618.72, ENTRY: 1622.62
Losses: 8.239999999999782

TP1: 1623.5700000000002, CLOSE: 1626.51, ENTRY: 1619.7
Profit: 27.450000000000045

TP2: 1619.7, CLOSE: 1627.49, ENTRY: 1619.7
Profit: 27.450000000000045

STOP LOSS: 1619.98, CLOSE: 1618.39, ENTRY: 1646.03
Losses: 34.289999999999736

wins: 2, profit: $27.450000000000045
losses: 4, losses: $34.289999999999736
P/L: -6.839999999999691


In [ ]:
# indicator = "atr"
# endpoint = f"https://api.taapi.io/{indicator}"
# parameters = {
#     'secret': ta_api,
#     'exchange': 'binance',
#     'symbol': 'ETH/USDT',
#     'interval': '15m'
#     } 
# response = requests.get(url = endpoint, params = parameters)
result = response.json() 
print(result['value'])